In [2]:
import os
from dotenv import load_dotenv
import urllib

# Load variables from .env file
load_dotenv()

server = os.getenv("SERVER")
database = os.getenv("DATABASE")
driver = os.getenv("DRIVER")

params = urllib.parse.quote_plus(
    f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
)

connection_string = f"mssql+pyodbc:///?odbc_connect={params}"
%sql $connection_string


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte